# Drifting y Quality

Un desafío común para los **data scientists** en el mundo real es que, a diferencia de las competencia de **Kaggle**, los datos cambian con el tiempo. Esto significa que los datos que recibiremos en el futuro no necesariamente reflejarán los que usamos para entrenar nuestros modelos. Esta homogeneidad es esencial, ya que los algoritmos aprenden patrones en función de las distribuciones observadas durante el entrenamiento. Si esas distribuciones cambian, la capacidad predictiva del modelo se deteriora.

Podemos identificar dos tipos principales de problemáticas asociadas a estos cambios:

- **Data Quality (DQ)**
- **Data Drift (DF)**

Comencemos con el primero: **Data Quality (DQ)**. Hay mucho que decir sobre este tema, pero mencionaremos brevemente que los datos provienen de procesos creados por seres humanos, quienes son propensos a cometer errores y hacer modificaciones. A lo largo del ciclo de vida de los modelos, es común que se enfrenten a este tipo de situaciones. Por ello, es fundamental implementar controles que garanticen que los datos a los que se aplican los modelos mantienen la misma estructura que los datos usados en el entrenamiento.

A continuación, veremos dos indicadores clave para monitorear la calidad de los datos, aunque no son los únicos a considerar.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

from os import path
# pd.set_option('display.max_rows', None)

# # Mostrar todas las columnas
# pd.set_option('display.max_columns', None)


c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [70]:
base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'
dataset_path = base_path + 'datasets/'
dataset_file = 'competencia_01.csv'

# agregue sus semillas
semillas = [540079, 250829, 314299, 302111, 801007]

data = pd.read_csv(dataset_path + dataset_file)

In [71]:
df=data.copy()

In [72]:
df=df.sort_values(by=['foto_mes', 'numero_de_cliente'])

## NaN por foto_mes

In [73]:
def calcular_porcentaje_nan_por_mes(dataset):
    # Crear una lista de las variables que quiero analizar los NaN
    variables = dataset.columns.difference(['numero_de_cliente', 'foto_mes', 'clase_ternaria'])

    # Inicializar una lista para almacenar los resultados
    resultados = []

    # Agrupar el dataset por 'foto_mes' y calcular el ratio de NaN para cada columna
    for mes, datos_mes in dataset.groupby('foto_mes'):
        # Calcular el porcentaje de NaN para cada columna en este mes
        porcentaje_nan = datos_mes[variables].isna().mean() * 100
        # Crear un DataFrame temporal para almacenar los resultados
        resultado_mes = pd.DataFrame({
            'foto_mes': mes,
            'campo': porcentaje_nan.index,
            'porcentaje_nan': porcentaje_nan.values
        })
        # Añadir los resultados del mes a la lista
        resultados.append(resultado_mes)

    # Concatenar todos los resultados en un único DataFrame
    tabla_resultados_nan = pd.concat(resultados, ignore_index=True)

    return tabla_resultados_nan

# Llamada a la función
tabla_porcentajes_nan_por_mes = calcular_porcentaje_nan_por_mes(df)

In [75]:
tabla_pivoteada_nan = tabla_porcentajes_nan_por_mes.pivot(index='campo', columns='foto_mes', values='porcentaje_nan')
tabla_pivoteada_nan['total_nan'] = tabla_pivoteada_nan.sum(axis=1)/6

# Ordenar la tabla por la columna 'total_nan' en orden descendente
tabla_ordenada_nan = tabla_pivoteada_nan.sort_values(by='total_nan', ascending=False)
tabla_ordenada_nan.head(20)

foto_mes,202101,202102,202103,202104,202105,202106,total_nan
campo,,,,,,,
Master_Finiciomora,99.631541,99.604663,99.310261,99.372905,99.493388,99.596060,99.501470
Visa_Finiciomora,97.367706,99.394390,99.036564,99.140106,97.336338,99.390451,98.610926
Master_cconsumos,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Master_cadelantosefectivo,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Master_mconsumosdolares,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Master_mconsumospesos,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Master_madelantopesos,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Master_madelantodolares,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Master_mpagosdolares,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142


In [76]:
col_eliminar_nan=["Master_Finiciomora","Visa_Finiciomora","Master_cconsumos","Master_cadelantosefectivo","Master_mconsumosdolares","Master_mconsumospesos",
                  "Master_madelantopesos","Master_madelantodolares","Master_mpagosdolares","Master_mpagospesos","Master_mconsumototal","Visa_fultimo_cierre","Master_fultimo_cierre"]

In [77]:
df.drop(columns=col_eliminar_nan, axis=1,inplace=True)

In [78]:
def calcular_porcentaje_ceros_por_mes(dataset):
    # Crear una lista de las variables que quiero analizar los ceros
    variables = dataset.columns.difference(['numero_de_cliente', 'foto_mes', 'clase_ternaria'])

    # Inicializar una lista para almacenar los resultados
    resultados = []

    # Agrupar el dataset por 'foto_mes' y calcular el ratio de ceros para cada columna
    for mes, datos_mes in dataset.groupby('foto_mes'):
        # Calcular el porcentaje de ceros para cada columna en este mes
        porcentaje_ceros = (datos_mes[variables] == 0).mean() * 100
        # Crear un DataFrame temporal para almacenar los resultados
        resultado_mes = pd.DataFrame({
            'foto_mes': mes,
            'campo': porcentaje_ceros.index,
            'porcentaje_ceros': porcentaje_ceros.values
        })
        # Añadir los resultados del mes a la lista
        resultados.append(resultado_mes)

    # Concatenar todos los resultados en un único DataFrame
    tabla_resultados = pd.concat(resultados, ignore_index=True)

    return tabla_resultados

# Llamada a la función
tabla_porcentajes_ceros_por_mes = calcular_porcentaje_ceros_por_mes(df)


In [79]:
tabla_pivoteada = tabla_porcentajes_ceros_por_mes.pivot(index='campo', columns='foto_mes', values='porcentaje_ceros')
tabla_pivoteada['total_zero'] = tabla_pivoteada.sum(axis=1)/6

# Ordenar la tabla por la columna 'total_zero' en orden descendente
tabla_pivoteada_zero = tabla_pivoteada.sort_values(by='total_zero', ascending=False)
tabla_pivoteada_zero.head(40)

foto_mes,202101,202102,202103,202104,202105,202106,total_zero
campo,,,,,,,
mcuenta_corriente_adicional,99.937047,99.937902,99.937074,99.937839,99.939255,99.940561,99.938280
cpayroll2_trx,99.875946,99.889330,99.904084,99.909806,99.907668,99.871419,99.893042
mpayroll2,99.875946,99.889330,99.904084,99.909806,99.907668,99.871419,99.893042
ccheques_emitidos_rechazados,99.777196,99.822313,99.829551,99.834847,99.831737,99.833815,99.821576
mcheques_emitidos_rechazados,99.777196,99.822313,99.829551,99.834847,99.831737,99.833815,99.821576
mcheques_depositados_rechazados,99.781517,99.835225,99.829551,99.803157,99.811691,99.793178,99.809053
ccheques_depositados_rechazados,99.781517,99.835225,99.829551,99.803157,99.811691,99.793178,99.809053
cliente_vip,99.678447,99.728859,99.729358,99.728198,99.727863,99.434727,99.671242
minversion1_dolares,99.620431,99.626797,99.631609,99.636175,99.640390,99.646401,99.633634


In [65]:
variables_zero= tabla_pivoteada_zero.head(40).index.tolist()

In [80]:
col_eliminar_zero= ['mcuenta_corriente_adicional',
 'mpayroll2',
 'mcheques_emitidos_rechazados',
 'mcheques_depositados_rechazados',
 'minversion1_dolares',
 'mforex_buy',
 'mplazo_fijo_pesos',
 'mpagodeservicios',
 'mprestamos_prendarios',
 'mprestamos_hipotecarios',
 'mcaja_ahorro_adicional',
 'mcheques_emitidos',
 'mcajeros_propios_descuentos',
 'minversion2',
 'minversion1_pesos',
 'mcheques_depositados',
 'mtarjeta_master_descuentos',
 'mforex_sell']

In [81]:
df.drop(columns=col_eliminar_zero, axis=1,inplace=True)

In [82]:
df.shape

(981946, 124)

In [83]:
ruta_datasets = "../../../datasets/"
nombre_exp="competencia_01_DQ.csv"
df.to_csv(ruta_datasets+nombre_exp, index=False)